In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random 
from matplotlib import pyplot as plt

from sklearn import decomposition
from sklearn import manifold
from scipy import stats

In [2]:
import sys
import utilities as ut

In [3]:
train_dataset, test_dataset = ut.load_files(dataset=1)
n = 5000#train_dataset.shape[0] 
epsilon = 0.1
input_size = train_dataset.shape[1]-1
hidden_size = 500
hidden_size2 = 500
num_classes = 2
num_epochs = 200
batch_size = 512
learning_rate = 0.001
learning_rate2 = 0.001
regularization = False
add_DR_based_data = True
train_dataset = train_dataset.sample(n)
samples = 5000

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0


In [4]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

In [6]:
train_dataset, test_dataset = ut.delete_outliers(train_dataset, test_dataset)

(3836, 61)
(28625, 60)
(28625, 61)


In [7]:
train_dataset = ut.sort_columns(train_dataset)
test_dataset = ut.sort_columns(test_dataset)

In [8]:
test_dataset_pred = test_dataset.copy()
train_dataset_pred = train_dataset.copy()

In [9]:
data_prior = ut.generate_samples(samples, train_dataset, epsilon,  option = 2)

In [10]:
train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.1, random_state=42)

In [11]:
train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.1, random_state=42)

In [12]:


def get_tensors(data, batch_size):
    data.loc[data.label=='ClassA','label'] = 1
    data.loc[data.label=='ClassB','label'] = 0
    data.loc[data.label=='Noise','label'] = 0.5
    target = torch.tensor(data['label'].values.astype(np.float32))
    x = torch.tensor(data.drop('label', axis = 1).values.astype(np.float32)) 
    x = f.normalize(x)
    xy = data_utils.TensorDataset(x, target) 
    xy = data_utils.DataLoader(dataset = xy, batch_size = batch_size)
    return data, x, target, xy


In [13]:
train_dataset_prior, train_prior, train_target_prior, train_loader_prior = get_tensors(train_dataset_prior, batch_size)
val_dataset_prior, val_prior, val_target_prior, val_loader_prior = get_tensors(val_dataset_prior, batch_size)
train_dataset, train, train_target, train_loader = get_tensors(train_dataset, batch_size)
train_dataset_pred, train_pred, train_target_pred, train_loader_pred = get_tensors(train_dataset_pred, batch_size)

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [14]:
#train_dataset_prior['label'] = train_dataset_prior['label'].str.replace('ClassA', '1')
#train_dataset_prior['label'] = train_dataset_prior['label'].str.replace('ClassB', '0')
#train_dataset_prior['label'] = train_dataset_prior['label'].str.replace('Noise', '0.5')
#train_target_prior = torch.tensor(train_dataset_prior['label'].values.astype(np.float32))
#train_prior = torch.tensor(train_dataset_prior.drop('label', axis = 1).values.astype(np.float32)) 
#train_prior = f.normalize(train_prior)
#train_tensor_prior = data_utils.TensorDataset(train_prior, train_target_prior) 
#train_loader_prior = data_utils.DataLoader(dataset = train_tensor_prior, batch_size = batch_size)

In [15]:
#val_dataset_prior['label'] = val_dataset_prior['label'].str.replace('ClassA', '1')
#val_dataset_prior['label'] = val_dataset_prior['label'].str.replace('ClassB', '0')
#val_dataset_prior['label'] = val_dataset_prior['label'].str.replace('Noise', '0.5')

#val_target_prior = torch.tensor(val_dataset_prior['label'].values.astype(np.float32))

#val_prior = torch.tensor(val_dataset_prior.drop('label', axis = 1).values.astype(np.float32)) 
#val_prior = f.normalize(val_prior)

#val_tensor_prior = data_utils.TensorDataset(val_prior, val_target_prior) 
#val_loader_prior = data_utils.DataLoader(dataset = val_tensor_prior, batch_size = batch_size)

Cases using DR 1: Period $ \in [0.2,1.0]$ days

In [16]:
#train_dataset['label'] = train_dataset['label'].str.replace('ClassA', '1')
#train_dataset['label'] = train_dataset['label'].str.replace('ClassB', '0')
#train_dataset['label'] = train_dataset['label'].str.replace('Noise', '0.5')
#train_target = torch.tensor(train_dataset['label'].values.astype(np.float32))
#train = torch.tensor(train_dataset.drop('label', axis = 1).values.astype(np.float32)) 
#train = f.normalize(train)
#train_tensor = data_utils.TensorDataset(train, train_target) 
#train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size)

In [17]:
#train_dataset_pred['label'] = train_dataset_pred['label'].str.replace('ClassA', '1')
#train_dataset_pred['label'] = train_dataset_pred['label'].str.replace('ClassB', '0')
#train_dataset_pred['label'] = train_dataset_pred['label'].str.replace('Noise', '0.5')
#train_target_pred = torch.tensor(train_dataset_pred['label'].values.astype(np.float32))
#train_pred = torch.tensor(train_dataset_pred.drop('label', axis = 1).values.astype(np.float32)) 
#train_pred = f.normalize(train_pred)
#train_tensor_pred = data_utils.TensorDataset(train_pred, train_target_pred) 
#train_loader_pred = data_utils.DataLoader(dataset = train_tensor_pred, batch_size = batch_size)

In [18]:
val_dataset['label'] = val_dataset['label'].str.replace('ClassA', '1')
val_dataset['label'] = val_dataset['label'].str.replace('ClassB', '0')
val_dataset['label'] = val_dataset['label'].str.replace('Noise', '0.5')
val_target = torch.tensor(val_dataset['label'].values.astype(np.float32))
val = torch.tensor(val_dataset.drop('label', axis = 1).values.astype(np.float32)) 
val = f.normalize(val)
val_tensor = data_utils.TensorDataset(val, val_target) 
val_loader = data_utils.DataLoader(dataset = val_tensor, batch_size = batch_size)

In [19]:
test_dataset['label'] = test_dataset['label'].str.replace('ClassA', '1')
test_dataset['label'] = test_dataset['label'].str.replace('ClassB', '0')
test_dataset['label'] = test_dataset['label'].str.replace('Noise', '0.5')
test_target = torch.tensor(test_dataset['label'].values.astype(np.float32))
test = torch.tensor(test_dataset.drop('label', axis = 1).values.astype(np.float32)) 
test = f.normalize(test)
test_tensor = data_utils.TensorDataset(test, test_target) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size)

In [20]:
test_dataset_pred['label'] = test_dataset_pred['label'].str.replace('ClassA', '1')
test_dataset_pred['label'] = test_dataset_pred['label'].str.replace('ClassB', '0')
test_dataset_pred['label'] = test_dataset_pred['label'].str.replace('Noise', '0.5')
test_target_pred = torch.tensor(test_dataset_pred['label'].values.astype(np.float32))
test_pred = torch.tensor(test_dataset_pred.drop('label', axis = 1).values.astype(np.float32)) 
test_pred = f.normalize(test_pred)
test_tensor_pred = data_utils.TensorDataset(test_pred, test_target_pred) 
test_loader_pred = data_utils.DataLoader(dataset = test_tensor_pred, batch_size = batch_size)

In [21]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size,hidden_size2, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        #self.relu = nn.ReLU()
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size2)  
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes) 
        #self.sigmoid = nn.Sigmoid()
        #self.dropout = nn.Dropout(p=0.1)
        #self.batchnorm1 = nn.BatchNorm1d(hidden_size)
        #self.batchnorm2 = nn.BatchNorm1d(hidden_size)
        
        
    def forward(self, x):
        out1 = self.fc1(x)
        #out = self.batchnorm1(out)
        #out = self.relu(out)
        out = self.relu(out1)
        out2 = self.fc2(out)
        #out = self.batchnorm1(out)
        out = self.relu2(out)
        out3 = self.fc3(out)
        #x = self.dropout(x)
        #out = self.sigmoid(out)
        return out3, out2, out1
  
#net_prior = Net(input_size, hidden_size, hidden_size2, num_classes)

#net_prior.cuda()

net = Net(input_size, hidden_size, hidden_size2, num_classes)

#use_cuda = torch.cuda.is_available()

net.cuda()
#net = nn.DataParallel(net)

Net(
  (fc1): Linear(in_features=60, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=500, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=500, out_features=2, bias=True)
)

In [22]:
param_l1 = list(net.fc1.parameters())
param_l2 = list(net.fc2.parameters())
param_l3 = list(net.fc3.parameters())
#print(len(param))
print(len(param_l1[0]))
print(len(param_l1[:][0]))
#print(len(param[1]))

500
500


In [23]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
#criterion = nn.BCELoss()
#criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  
optimizer_prior = torch.optim.Adam(net.parameters(), lr=learning_rate)   

In [ ]:
from itertools import cycle
aux_loss_activated = True
# Train the Model   
loss_prior = torch.tensor(0)
hist_train = []
hist_val = []
aux_loss_behaviour = []
num_epochs_prior = 2000


EPS = 1e-3
locked_masks = {n: torch.abs(w) < EPS for n, w in net.named_parameters() if n.endswith('weight')}

EPS = 1e-6
locked_masks2 = {n: torch.abs(w) < EPS for n, w in net.named_parameters() if n.endswith('weight')}


for epoch in range(num_epochs_prior):
    epoch_loss = 0.0
    running_loss = 0.0
    epoch_loss_prior = 0.0    
    running_loss_prior = 0.0
    
    for item1, item2 in zip(train_loader, cycle(train_loader_prior)):
        star_prior, labels_prior = item2
        star, labels = item1
        
        star = Variable(star.view(-1, input_size)).cuda()
        labels = Variable(labels).cuda()
        optimizer.zero_grad()  
        outputs, _, _ = net(star)
        loss = criterion(outputs, labels.long())      
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        if aux_loss_activated:
            star_prior = Variable(star_prior.view(-1, input_size)).cuda()
            labels_prior = Variable(labels_prior).cuda()
            optimizer_prior.zero_grad()  # zero the gradient buffer
            outputs_prior, _, _ = net(star_prior)
            #print("-------------------before----------------------")
            aux_loss_behaviour.append(loss_prior.item())
            loss_prior = criterion(outputs_prior, labels_prior.long()) #custom_loss_auxiliar(outputs_prior)      
            #print("-------------------later-----------------------")
            aux_loss_behaviour.append(loss_prior.item())
            loss_prior.backward()
            
            for n, w in net.named_parameters():                                                                                                                                                                           
                if w.grad is not None and n in locked_masks2:                                                                                                                                                                                   
                    w.grad[locked_masks2[n]] = 0 
                
                
            optimizer_prior.step()
            epoch_loss_prior += outputs_prior.shape[0] * loss_prior.item()      
            running_loss_prior += loss_prior.item()
        
    hist_train.append(running_loss/len(train_loader))    
    print('training:', 'epoch: ', str(epoch+1),' loss: ', str(running_loss/len(train_loader)))
    
    epoch_loss = 0.0
    running_loss = 0.0
    for i, (star, labels) in enumerate(val_loader):  
        
        star = Variable(star.view(-1, input_size)).cuda()
        labels = Variable(labels).cuda()
        optimizer.zero_grad()  
        outputs, _, _ = net(star)
        loss = criterion(outputs, labels.long())      
        loss.backward()
        
        for n, w in net.named_parameters():                                                                                                                                                                           
            if w.grad is not None and n in locked_masks:                                                                                                                                                                                   
                w.grad[locked_masks[n]] = 0 
        
        optimizer.step()
        running_loss += loss.item()
    
    print('validating:', 'epoch: ', str(epoch+1),' loss: ', str(running_loss / len(val_loader)))
    hist_val.append(running_loss / len(val_loader))

training: epoch:  1  loss:  0.6088828699929374
validating: epoch:  1  loss:  0.458293080329895
training: epoch:  2  loss:  0.3849938086100987
validating: epoch:  2  loss:  0.28324228525161743
training: epoch:  3  loss:  0.2859543945108141
validating: epoch:  3  loss:  0.22534872591495514
training: epoch:  4  loss:  0.2753621850694929
validating: epoch:  4  loss:  0.221796914935112
training: epoch:  5  loss:  0.2766495943069458
validating: epoch:  5  loss:  0.21786566078662872
training: epoch:  6  loss:  0.2645669941391264
validating: epoch:  6  loss:  0.20905093848705292
training: epoch:  7  loss:  0.2485556368316923
validating: epoch:  7  loss:  0.2019096165895462
training: epoch:  8  loss:  0.23630546246256148
validating: epoch:  8  loss:  0.19768132269382477
training: epoch:  9  loss:  0.22785756843430655
validating: epoch:  9  loss:  0.19376957416534424
training: epoch:  10  loss:  0.22105399199894496
validating: epoch:  10  loss:  0.18947727978229523
training: epoch:  11  loss:  0

training: epoch:  85  loss:  0.06856824297990118
validating: epoch:  85  loss:  0.07130677253007889
training: epoch:  86  loss:  0.06800505199602672
validating: epoch:  86  loss:  0.07059437036514282
training: epoch:  87  loss:  0.06744267738291196
validating: epoch:  87  loss:  0.06990179419517517
training: epoch:  88  loss:  0.06690363852041108
validating: epoch:  88  loss:  0.06918712705373764
training: epoch:  89  loss:  0.0663633192224162
validating: epoch:  89  loss:  0.06852350383996964
training: epoch:  90  loss:  0.0658413517688002
validating: epoch:  90  loss:  0.06784636527299881
training: epoch:  91  loss:  0.06532793013112885
validating: epoch:  91  loss:  0.06716706603765488
training: epoch:  92  loss:  0.06480919090764863
validating: epoch:  92  loss:  0.06651095300912857
training: epoch:  93  loss:  0.06431614501135689
validating: epoch:  93  loss:  0.06585962325334549
training: epoch:  94  loss:  0.06382538484675544
validating: epoch:  94  loss:  0.06520432233810425
tr

validating: epoch:  168  loss:  0.03503022715449333
training: epoch:  169  loss:  0.040352960782391686
validating: epoch:  169  loss:  0.03477280214428902
training: epoch:  170  loss:  0.04014876378434045
validating: epoch:  170  loss:  0.03453395888209343
training: epoch:  171  loss:  0.03994046417730195
validating: epoch:  171  loss:  0.034280065447092056
training: epoch:  172  loss:  0.03973887275372233
validating: epoch:  172  loss:  0.03404352441430092
training: epoch:  173  loss:  0.03953418933919498
validating: epoch:  173  loss:  0.03380567952990532
training: epoch:  174  loss:  0.03933891387922423
validating: epoch:  174  loss:  0.03358205035328865
training: epoch:  175  loss:  0.03914256766438484
validating: epoch:  175  loss:  0.03331676125526428
training: epoch:  176  loss:  0.038943437061139514
validating: epoch:  176  loss:  0.03309756889939308
training: epoch:  177  loss:  0.03874596740518298
validating: epoch:  177  loss:  0.032873183488845825
training: epoch:  178  los

training: epoch:  251  loss:  0.027705866045185497
validating: epoch:  251  loss:  0.020899802446365356
training: epoch:  252  loss:  0.027595762429492816
validating: epoch:  252  loss:  0.020800800994038582
training: epoch:  253  loss:  0.02747936945940767
validating: epoch:  253  loss:  0.020672863349318504
training: epoch:  254  loss:  0.027369223002876555
validating: epoch:  254  loss:  0.020562494173645973
training: epoch:  255  loss:  0.02726526079433305
validating: epoch:  255  loss:  0.020458921790122986
training: epoch:  256  loss:  0.02716459839471749
validating: epoch:  256  loss:  0.020328659564256668
training: epoch:  257  loss:  0.027049209656459943
validating: epoch:  257  loss:  0.020226001739501953
training: epoch:  258  loss:  0.026941959879228046
validating: epoch:  258  loss:  0.020119143649935722
training: epoch:  259  loss:  0.026842460302369937
validating: epoch:  259  loss:  0.019999397918581963
training: epoch:  260  loss:  0.026741248422435353
validating: epoc

training: epoch:  334  loss:  0.020654873124190738
validating: epoch:  334  loss:  0.014008459635078907
training: epoch:  335  loss:  0.020589412855250493
validating: epoch:  335  loss:  0.013944442383944988
training: epoch:  336  loss:  0.020529443664210185
validating: epoch:  336  loss:  0.013892349787056446
training: epoch:  337  loss:  0.020466962030955722
validating: epoch:  337  loss:  0.013830105774104595
training: epoch:  338  loss:  0.020405701228550503
validating: epoch:  338  loss:  0.013776673935353756
training: epoch:  339  loss:  0.02034042589366436
validating: epoch:  339  loss:  0.013719345442950726
training: epoch:  340  loss:  0.020279870634632453
validating: epoch:  340  loss:  0.013664106838405132
training: epoch:  341  loss:  0.02022302057594061
validating: epoch:  341  loss:  0.013606597669422626
training: epoch:  342  loss:  0.020158982436571802
validating: epoch:  342  loss:  0.013560682535171509
training: epoch:  343  loss:  0.020097170025110245
validating: epo

...

training: epoch:  1302  loss:  0.003465208723875029
validating: epoch:  1302  loss:  0.0022690342739224434
training: epoch:  1303  loss:  0.0034543410542287995
validating: epoch:  1303  loss:  0.0022549324203282595
training: epoch:  1304  loss:  0.0034532296537820783
validating: epoch:  1304  loss:  0.002248970326036215
training: epoch:  1305  loss:  0.003447404463908502
validating: epoch:  1305  loss:  0.0022427490912377834
training: epoch:  1306  loss:  0.003440096209357892
validating: epoch:  1306  loss:  0.0022394026163965464
training: epoch:  1307  loss:  0.003438401667933379
validating: epoch:  1307  loss:  0.0022608309518545866
training: epoch:  1308  loss:  0.003423062286206654
validating: epoch:  1308  loss:  0.002227942692115903
training: epoch:  1309  loss:  0.0034254312215905103
validating: epoch:  1309  loss:  0.0022270968183875084
training: epoch:  1310  loss:  0.0034221196048227803
validating: epoch:  1310  loss:  0.00223760143853724
training: epoch:  1311  loss:  0.0034

training: epoch:  1381  loss:  0.003100725738996906
validating: epoch:  1381  loss:  0.0019797475542873144
training: epoch:  1382  loss:  0.0030861686183405773
validating: epoch:  1382  loss:  0.0019469467224553227
training: epoch:  1383  loss:  0.0030864628248049746
validating: epoch:  1383  loss:  0.0019561999943107367
training: epoch:  1384  loss:  0.003082376846577972
validating: epoch:  1384  loss:  0.0019619851373136044
training: epoch:  1385  loss:  0.0030707537246468875
validating: epoch:  1385  loss:  0.0019199554808437824
training: epoch:  1386  loss:  0.003079910155585302
validating: epoch:  1386  loss:  0.0019674403592944145
training: epoch:  1387  loss:  0.0030630750448576043
validating: epoch:  1387  loss:  0.0019393753027543426
training: epoch:  1388  loss:  0.003060602067437555
validating: epoch:  1388  loss:  0.0019273321377113461
training: epoch:  1389  loss:  0.003064335764585329
validating: epoch:  1389  loss:  0.0019501220667734742
training: epoch:  1390  loss:  0.

training: epoch:  1462  loss:  0.0027669722413910286
validating: epoch:  1462  loss:  0.0017035002820193768
training: epoch:  1463  loss:  0.0027617229581145303
validating: epoch:  1463  loss:  0.0016931857680901885
training: epoch:  1464  loss:  0.0027616729494184256
validating: epoch:  1464  loss:  0.001689961296506226
training: epoch:  1465  loss:  0.002757187733160598
validating: epoch:  1465  loss:  0.0016993583412840962
training: epoch:  1466  loss:  0.0027481707012546913
validating: epoch:  1466  loss:  0.0016826745122671127
training: epoch:  1467  loss:  0.002746902871876955
validating: epoch:  1467  loss:  0.0016731792129576206
training: epoch:  1468  loss:  0.002745217943031873
validating: epoch:  1468  loss:  0.0016939375782385468
training: epoch:  1469  loss:  0.0027363276458345354
validating: epoch:  1469  loss:  0.0016648373566567898
training: epoch:  1470  loss:  0.002736573581517275
validating: epoch:  1470  loss:  0.0016750921495258808
training: epoch:  1471  loss:  0.

training: epoch:  1540  loss:  0.0024882420880853067
validating: epoch:  1540  loss:  0.001470879535190761
training: epoch:  1541  loss:  0.0024877548816480805
validating: epoch:  1541  loss:  0.0014639394357800484
training: epoch:  1542  loss:  0.0024865782220980953
validating: epoch:  1542  loss:  0.0014708932721987367
training: epoch:  1543  loss:  0.00248100542064224
validating: epoch:  1543  loss:  0.0014571683714166284
training: epoch:  1544  loss:  0.002478129097393581
validating: epoch:  1544  loss:  0.0014584112213924527
training: epoch:  1545  loss:  0.0024763523833826184
validating: epoch:  1545  loss:  0.0014719926984980702
training: epoch:  1546  loss:  0.002470799979554223
validating: epoch:  1546  loss:  0.0014425615081563592
training: epoch:  1547  loss:  0.002469169200464551
validating: epoch:  1547  loss:  0.0014498679665848613
training: epoch:  1548  loss:  0.002464616596366146
validating: epoch:  1548  loss:  0.0014561815187335014
training: epoch:  1549  loss:  0.00

training: epoch:  1619  loss:  0.002246940658161683
validating: epoch:  1619  loss:  0.0012629381380975246
training: epoch:  1620  loss:  0.0022389853256754577
validating: epoch:  1620  loss:  0.0012438312405720353
training: epoch:  1621  loss:  0.002240508753207645
validating: epoch:  1621  loss:  0.001262599602341652
training: epoch:  1622  loss:  0.0022335135893497083
validating: epoch:  1622  loss:  0.0012498401338234544
training: epoch:  1623  loss:  0.002231406937685928
validating: epoch:  1623  loss:  0.001244367565959692
training: epoch:  1624  loss:  0.0022297475703193675
validating: epoch:  1624  loss:  0.0012516146525740623
training: epoch:  1625  loss:  0.0022241601504252423
validating: epoch:  1625  loss:  0.0012411401839926839
training: epoch:  1626  loss:  0.002223959575141115
validating: epoch:  1626  loss:  0.0012449893401935697
training: epoch:  1627  loss:  0.002221540838945657
validating: epoch:  1627  loss:  0.001240518526174128
training: epoch:  1628  loss:  0.002

In [ ]:
plt.plot(aux_loss_behaviour)
plt.show()

In [ ]:
# Test the Model
correct = 0
total = 0
for star, labels in test_loader:
    images = Variable(star.view(-1, input_size)).cuda()
    outputs, _, _ = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels.long()).sum()
print('Accuracy of the network on test objects: %d %%' % (100 * correct / total))
acc_testing = 100 *correct / total
print(np.asarray(acc_testing))

In [ ]:
correct = 0
total = 0
for star, labels in train_loader:
    images = Variable(star.view(-1, input_size)).cuda()
    outputs, _, _ = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels.long()).sum()
print('Accuracy of the network on train objects: %d %%' % (100 * correct / total))
acc_training = 100 *correct / total

In [ ]:
# Save the Model
torch.save(net.state_dict(), 'model.pkl')

In [ ]:
plt.plot(hist_val, label ='validation')
plt.plot(hist_train, label ='train')
plt.legend()
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.savefig('images/'+str(samples)+'_'+str(epsilon)+'_'+str(n)+"_"+str(hidden_size)+"_Loss_Training.png")
plt.show()

In [ ]:
csv_file = open("size_MLP_noise.csv", "a")
csv_file.write(str(np.asarray(acc_testing))+","+str(np.asarray(acc_training))+","+str(samples)+","+str(epsilon)+","+str(n)+","+str(hidden_size)+"\n")
csv_file.close()

In [ ]:
outputs, intermediates, intermediates2, labels = ut.get_representations(net, train_loader, device)
outputs_test, intermediates_test, intermediates2_test, labels_test = ut.get_representations(net, test_loader, device)

In [ ]:
intermediate_pca_data, intermediate_pca_data_test = ut.get_pca(intermediates, data_test=intermediates_test)
fig, axs = plt.subplots(2, 3, figsize=(20,15))
ut.plot_representations(intermediate_pca_data, labels, axs[0, 0])
ut.plot_representations(intermediate_pca_data_test, labels_test, axs[1, 0])
intermediate2_pca_data, intermediate2_pca_data_test = ut.get_pca(intermediates2, data_test=intermediates2_test)
ut.plot_representations(intermediate2_pca_data, labels, axs[0, 1])
ut.plot_representations(intermediate2_pca_data_test, labels_test, axs[1, 1])
output_pca_data, output_pca_data_test = ut.get_pca(outputs, data_test=outputs_test)
ut.plot_representations(output_pca_data, labels, axs[0, 2])
ut.plot_representations(output_pca_data_test, labels_test, axs[1, 2])

In [ ]:
N_CURVES = 25000

fig, axs = plt.subplots(2, 3, figsize=(20,15))
intermediate_tsne_data, intermediate_tsne_data_test = ut.get_tsne(intermediates, data_test= intermediates_test, n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data, labels, axs[0, 0],  n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data_test, labels_test, axs[1, 0], n_curves = N_CURVES)

intermediate2_tsne_data, intermediate2_tsne_data_test = ut.get_tsne(intermediates2, data_test=intermediates2_test, n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data, labels, axs[0, 1], n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data_test, labels_test, axs[1, 1], n_curves = N_CURVES)

output_tsne_data, output2_tsne_data_test = ut.get_tsne(outputs, data_test=outputs_test, n_curves = N_CURVES)
ut.plot_representations(output_tsne_data, labels, axs[0, 2], n_curves = N_CURVES)
ut.plot_representations(output2_tsne_data_test, labels_test, axs[1, 2], n_curves = N_CURVES)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,15))
curves, labels, probs_train = ut.get_predictions(net, train_loader_pred, device)
pred_labels = probs_train.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax1)
curves, labels, probs_test = ut.get_predictions(net, test_loader_pred, device)
pred_labels = probs_test.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax2)

In [ ]:
print(len(train_tensor))
print(len(train_tensor_pred))

In [ ]:
curves, labels, probs_train_sample = ut.get_predictions(net, train_loader, device)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(probs_train[:,0], color='black')
ax1.set_xlabel('training set')
ax2.hist(probs_train_sample[:,0], color='black')
ax2.set_xlabel('training set + samples')
ax3.hist(probs_test[:,0], color='black')
ax3.set_xlabel('testing set')
plt.show()

In [ ]:
#N_WEIGHTS = 25
#weights = net.fc2.weight.data
#plot_weights(weights, N_WEIGHTS)

In [ ]:
weights1 = net.fc1.weight.data
w1 = weights1.cpu().numpy().reshape(-1,1)
weights2 = net.fc2.weight.data
w2 = weights2.cpu().numpy().reshape(-1,1)
weights3 = net.fc3.weight.data
w3 = weights3.cpu().numpy().reshape(-1,1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(w1, color='black')
ax1.set_xlabel('Layer 1')
ax2.hist(w2, color='black')
ax2.set_xlabel('Layer 2')
ax3.hist(w3, color='black')
ax3.set_xlabel('Layer 3')
plt.show()